Carteira baseada no número de Graham sobre o valor intrínseco de uma ação, foca em pegar empresas que sejam lucrativas e baratas: empresas operando a P/L menor que 15 e com P/VPA menor que 1,5. Multiplicando 15 x 1,5 temos o número 22,5 que nos indica a fórmula de graham. Valor Intrínseco de uma ação = $\sqrt{22,5 \times \text{LPA} \times \text{VPA}}$.

Critérios:


- Ter Lucro por ação maior que zero ,isto é, a empresa não pode estar com prejuízo atualmente
- Ter Valor Patrimonial por ação positivo, ou seja, a empresa não pode ter mais passivos (obrigações a pagar) que ativos (bens ou direitos a receber)
- Ter volume médio de negociação diário de no mínimo R$250.000,00
- Ter lucro líquido medio positivo em todos os últimos 5 exercícios.


In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from IPython.display import display, HTML

def b_print(df , n=30 , clean=True): #beauty print :)
    
    # from IPython.display import display, HTML

    # if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
    #     df['prefixo'] = df['Papel'].astype(str).str[:4]
    #     df=df.drop_duplicates(subset='prefixo', keep='first')
    #     # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None


In [2]:

# atualiza info fundamentlista

from DT_atualiza_settings import *
from DT_StatusInvest import SI
SI(mercado = 'Acoes' )

 ====== SI Acoes ===== 
 ====== Busca
 ====== Download 


In [41]:

#carrega a info fundamentalista num dataframe

import os
# Caminho do arquivo local
file_path = os.path.expanduser('~/GHub/Finance-playground/data/SI_Acoes.csv')
# URL para o arquivo online
file_url = 'https://raw.githubusercontent.com/BDonadelli/Finance-playground/refs/heads/main/data/SI_Acoes.csv'

# Verificar se o arquivo existe localmente
if os.path.exists(file_path):
    # Ler o arquivo local
    dados = pd.read_csv(file_path,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido localmente.")
else:
    # Ler o arquivo a partir da URL
    dados = pd.read_csv(file_url,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido da URL, pode não estar autalizado.")


Arquivo lido localmente.


In [42]:
dados.columns = dados.columns.str.strip()
dados.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', 'LIQUIDEZ MEDIA DIARIA',
       'VPA', 'LPA', 'PEG Ratio', 'VALOR DE MERCADO'],
      dtype='object')

In [43]:
cols = [
    'TICKER', 'PRECO',
    'valor intrinseco', 'Delta (%)',
    'CAGR LUCROS 5 ANOS',
    'P/L', 'P/VP',
    'ROE', 'DIVIDA LIQUIDA / EBIT',
]

In [44]:
# criterios basicos de seleção

criterios = (
        (dados['LIQUIDEZ MEDIA DIARIA'] > 300000) &
        (dados['LPA'] > 0) & 
        (dados['VPA'] > 0) & 
        (dados['CAGR LUCROS 5 ANOS'] > 0)
)



# 'DIVIDA LIQUIDA / EBIT' é NaN para bancos

bancos = [
    "ABCB4",
    "BAZA3", "BBAS3", "BBDC3", "BBDC4", "BEES3", "BEES4", "BGIP3", "BGIP4",
    "BGIP4", "BMEB3", "BMEB4", "BMGB4", "BNBR3", "BPAC11", "BPAC3", "BPAC5", "BPAN11", "BPAN4", "BPAR3",
    "BPAT33", "BRSR3", "BRSR5", "BRSR6", "BSLI3", "BSLI4", "BSLI4", "BRIV3", "BRIV4", "BMIN3", "BMIN4"
    "IRBR3", "ITUB3", "ITUB4", 
    "MERC3", "MERC4", "MODL3",  "MODL4", "MODL11",  "PINE3","PINE4", "SANB11", "SANB3","SANB4",
    "BBSE3", "CRIV3", "CRIV4", "CSAB3", "CSAB4", "FIGE3", "FIGE4", "FIGE3", "FNCN3", "IDVL3", "IDVL4" , "IRBR3",
    "BMIN3" , "BMIN4", "BIDI3", "BIDI4", "BIDI11", 
    ]

# Substitui por 0 quando o ticker for de banco
dados.loc[dados["TICKER"].isin(bancos),    "DIVIDA LIQUIDA / EBIT"] = 0

# criterios com filtros extras de qualidade

criterios_extras = (
    (criterios) & 
    (dados['ROE'] > 0.08) &
    (dados['DIVIDA LIQUIDA / EBIT'] < 3.0)
)

In [46]:

# retira linha de ticker que não são mais negociados

delisted = [
    'SOMA3'
]
dados = dados[~dados['TICKER'].isin(delisted)]


In [47]:
# valor intrinseco da acao
dados['valor intrinseco'] = np.round(np.sqrt(22.5 * dados['LPA'] * dados['VPA']),2)
# potencial de crescimento 
dados['Delta (%)'] = np.round((dados['valor intrinseco'] / dados['PRECO'] -1)*100,2)
'''
rank() atribui uma posição relativa a cada valor da série. O menor valor recebe rank 1 (ascending=True)
Todos os valores empatados recebem o menor rank possível dentro do empate (method="min")
'''
dados["Rank"] = dados['Delta (%)'].rank(ascending=True, method="min")
# ordena por rank em ordem decrescente
dados.sort_values(by="Rank", ascending=False, inplace=True)

# ajeitando uns numeros

dados['VAL DE MERCADO (em Bilhoes)'] = dados['VALOR DE MERCADO']  / 1e9
dados['LIQ MEDIA DIARIA (em Milhoes)'] = dados['LIQUIDEZ MEDIA DIARIA'] / 1e6


In [48]:
b_print(dados.loc[criterios, cols],n=10)

TICKER,PRECO,valor intrinseco,Delta (%),CAGR LUCROS 5 ANOS,P/L,P/VP,ROE,DIVIDA LIQUIDA / EBIT
LIGT3,4.90,45.73,833.27,11.57,0.80,0.32,40.83,5.10
ALLD3,8.47,38.62,355.96,28.55,2.28,0.48,20.88,-0.32
BAZA3,78.00,236.60,203.33,31.27,4.07,0.60,14.74,0.00
COGN3,3.33,9.43,183.18,41.43,5.16,0.54,10.43,3.26
BRSR6,16.19,43.58,169.18,0.02,5.21,0.60,11.45,0.00
MTRE3,3.69,9.84,166.67,8.93,7.96,0.39,4.95,5.80
TASA4,5.13,13.64,165.89,23.30,5.77,0.55,9.56,3.87
EUCA4,18.20,46.61,156.10,35.10,5.70,0.60,10.54,0.99
LOGG3,25.63,65.49,155.52,33.60,5.87,0.59,10.02,3.28
JHSF3,7.92,19.78,149.75,32.04,4.19,0.86,20.57,1.94


In [49]:
b_print(dados.loc[criterios_extras, cols],n=11)

TICKER,PRECO,valor intrinseco,Delta (%),CAGR LUCROS 5 ANOS,P/L,P/VP,ROE,DIVIDA LIQUIDA / EBIT
ALLD3,8.47,38.62,355.96,28.55,2.28,0.48,20.88,-0.32
BAZA3,78.00,236.60,203.33,31.27,4.07,0.60,14.74,0.00
BRSR6,16.19,43.58,169.18,0.02,5.21,0.60,11.45,0.00
EUCA4,18.20,46.61,156.10,35.10,5.70,0.60,10.54,0.99
JHSF3,7.92,19.78,149.75,32.04,4.19,0.86,20.57,1.94
RECV3,11.08,27.17,145.22,57.65,5.24,0.71,13.59,1.75
PETR4,29.64,66.61,124.73,14.07,4.93,0.90,18.33,2.41
MELK3,3.72,8.26,122.04,21.01,6.50,0.70,10.75,1.29
PETR3,31.15,66.61,113.84,14.07,5.18,0.95,18.33,2.41
GMAT3,4.49,9.25,106.01,41.00,5.37,0.99,18.46,0.59


In [50]:

# # remove empresas repetidas, mantem primeira ocorrencia

# dados['prefixo'] = dados['TICKER'].str[:4]
# # dados.drop_duplicates(subset='prefixo', keep='first')
# dados_limpo = dados.drop_duplicates(subset='prefixo', keep='first')
# # df_limpo = df_limpo.drop('prefixo', axis=1) 
# b_print(dados_limpo[colunas_exibidas])

In [51]:

# Filtro de valor (Graham + margem)
value_filter = dados['Delta (%)'] >= 25
# Catalisador de valuation — P/L deprimido
catalyst_pl = dados['P/L'] < 12.0
# Catalisador patrimonial — P/VPA baixo
catalyst_pvp = dados['P/VP'] < 1.2
# Pelo menos 1 catalisador ativo
dados['Catalisador'] = (
    catalyst_pl |
    catalyst_pvp
)

# Screening final 

screening = dados[
    criterios_extras &
    value_filter &
    dados['Catalisador']
].copy()
# Score de position trade (simples e robusto)
screening['Score'] = (
    (screening['Delta (%)'] / 100) * 0.5 +
    (screening['CAGR LUCROS 5 ANOS'].clip(0, 20) / 100) * 0.3 +
    ((12 - screening['P/L']).clip(0, 12) / 12) * 0.2
)
#  Ordenação final
screening.sort_values('Score', ascending=False, inplace=True)
screening.reset_index(drop=True, inplace=True)
# screening.index = screening.index + 1


cols = [
    'TICKER', 'PRECO',
    'valor intrinseco', 'Delta (%)',
    'CAGR LUCROS 5 ANOS',
    'P/L', 'P/VP',
    'ROE', 'DIVIDA LIQUIDA / EBIT',
    'Score'
]

b_print(screening[cols],n=20)

TICKER,PRECO,valor intrinseco,Delta (%),CAGR LUCROS 5 ANOS,P/L,P/VP,ROE,DIVIDA LIQUIDA / EBIT,Score
ALLD3,8.47,38.62,355.96,28.55,2.28,0.48,20.88,-0.32,2.001800
BAZA3,78.00,236.60,203.33,31.27,4.07,0.60,14.74,0.00,1.208817
BRSR6,16.19,43.58,169.18,0.02,5.21,0.60,11.45,0.00,0.959127
EUCA4,18.20,46.61,156.10,35.10,5.70,0.60,10.54,0.99,0.945500
JHSF3,7.92,19.78,149.75,32.04,4.19,0.86,20.57,1.94,0.938917
RECV3,11.08,27.17,145.22,57.65,5.24,0.71,13.59,1.75,0.898767
PETR4,29.64,66.61,124.73,14.07,4.93,0.90,18.33,2.41,0.783693
MELK3,3.72,8.26,122.04,21.01,6.50,0.70,10.75,1.29,0.761867
PETR3,31.15,66.61,113.84,14.07,5.18,0.95,18.33,2.41,0.725077
GMAT3,4.49,9.25,106.01,41.00,5.37,0.99,18.46,0.59,0.700550


Interpretação prática (position trade)

Delta alto + Score alto
→ desconto real com gatilho

Score alto, Delta médio
→ re-rating provável

Delta alto, Score baixo
→ possível value trap

================================================================================================

In [53]:
b_print(screening[['TICKER']],n=15)

TICKER
ALLD3
BAZA3
BRSR6
EUCA4
JHSF3
RECV3
PETR4
MELK3
PETR3
GMAT3
